###Email Spam Detection###

In [ ]:
#import libraries for different purposes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix , recall_score , precision_score
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Download the stopwords dataset

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load the dataset
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [ ]:
data = pd.read_csv('/content/spam.csv', encoding='latin-1')

In [ ]:
#finding null values
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [ ]:
data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [ ]:
data.shape

(5572, 5)

In [ ]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
#dropping null values
data1 = data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis=1)

In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
data1

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
data1.rename(columns = {"v1" : "Status", "v2":"Message"},inplace = True)
data1.head()

,Status,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
encoder = LabelEncoder()
data1['Status'] = encoder.fit_transform(data1['Status'])

In [ ]:
data1.describe()

,Status
count,5572.000000
mean,0.134063
std,0.340751
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
data1.columns

Index(['Status', 'Message'], dtype='object')

In [ ]:
data1.head()

,Status,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#Find duplicates
data1.duplicated().sum()


403

In [ ]:
#drop duplicates

data1 = data1.drop_duplicates(keep = 'first')
data1.duplicated().sum()


0

In [ ]:
# Select the desired columns using their indices
X = data1.iloc[:, 1]  # Select the first column
y = data1.iloc[:, 0]  # Select the second column

In [ ]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5169, dtype: object


In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5169,)
(4179,)
(1393,)


In [ ]:
print(y.shape)
print(y_train.shape)
print(y_test.shape)

(5169,)
(4179,)
(1393,)


In [ ]:
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [ ]:
# Create a TF-IDF vectorizer to convert text messages into numerical features

feature_extraction = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True)

In [ ]:
# Convert the text data into a matrix of token counts
cv = CountVectorizer()
X_train_count = cv.fit_transform(X_train.values)
X_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# Convert the training and testing text messages into numerical features using TF-IDF

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [ ]:
# Convert the target values to integers (0 and 1)

y_train = y_train.astype("int")
y_test = y_test.astype("int")

In [ ]:
print(X_train_features)

  (0, 4043)	0.3756574091465328
  (0, 1157)	0.45668413584121637
  (0, 6688)	0.28424126124827975
  (0, 3004)	0.34527827472369876
  (0, 6484)	0.5439043247529121
  (0, 7168)	0.3930381851005929
  (1, 3865)	0.22630968926545644
  (1, 3692)	0.26254280571277844
  (1, 3857)	0.21431237877715376
  (1, 2827)	0.1459966313547371
  (1, 6577)	0.3848850369442057
  (1, 797)	0.20894933806300164
  (1, 1730)	0.2571797649986263
  (1, 7292)	0.3007645147561468
  (1, 3594)	0.2766493012883344
  (1, 6467)	0.23306455153081398
  (1, 3106)	0.20020588320159777
  (1, 4678)	0.2306642275154712
  (1, 3291)	0.2688859365588395
  (1, 3241)	0.2766493012883344
  (1, 3021)	0.14766358006775415
  (1, 7223)	0.19925264826120356
  (1, 4638)	0.17242508268759021
  (2, 4828)	0.2564604294891203
  (2, 5785)	0.3574452045773296
  :	:
  (4177, 5865)	0.25608279848653
  (4177, 4938)	0.24407197045798038
  (4177, 1291)	0.2289401167005588
  (4177, 3648)	0.22353932432469437
  (4177, 2394)	0.22353932432469437
  (4177, 5936)	0.2150175062199579
  (

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_count, y_train)


RandomForestClassifier()

In [ ]:
from sklearn.metrics import confusion_matrix , recall_score , precision_score
from sklearn.metrics import accuracy_score

In [ ]:
#testing the spam
mail_ham = ['Same. Wanna plan a trip sometime then']
mail_ham_count = cv.transform(mail_ham)

y_pred = model.predict(mail_ham_count)
y_pred_proba = model.predict_proba(mail_ham_count)


print("Prediction:", y_pred[0])
print("Probabilities (Ham, Spam):", y_pred_proba[0])


Prediction: 0
Probabilities (Ham, Spam): [1. 0.]


In [ ]:
#finding accuracy of the training dataset
model.score(X_train_count,y_train)

1.0

In [ ]:
#finding accuracy of the test dataset
X_test_count = cv.transform(X_test)
model.score(X_test_count,y_test)

0.9784637473079684

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_count, y_train)

LogisticRegression()

In [ ]:
model = SVC(probability=True)
model.fit(X_train_count, y_train)

SVC(probability=True)

In [ ]:
#using SVM
from sklearn.svm import SVC
model = SVC()
model.fit(X_train_count, y_train)


SVC()

In [ ]:
from sklearn.metrics import confusion_matrix , recall_score , precision_score
from sklearn.metrics import accuracy_score

In [ ]:
#Testing  the mail (spam/ham)

mail_ham = ['Go until jurong point, crazy.. Available only ...	']
mail_ham_count = cv.transform(mail_ham)
y_pred = model.predict(mail_ham_count)
y_pred

array([0])

In [ ]:
#finding accuracy of the training dataset
model.score(X_train_count,y_train)

0.9964106245513281

In [ ]:
#finding accuracy of the test dataset
X_test_count = cv.transform(X_test)
model.score(X_test_count,y_test)

0.9806173725771715

Thankyou